# Evasion attack against a DL-based Intrusion Detection System
In this laboratory, you will train a neural network on a dataset of benign and DDoS attack network traffic.
Once the training process is completed, you will test the resulting model on unseen test data to evaluate the performance of the model on benign and malicious data. 

The model will be then tested on test data which has been artificially perturbed by adding some Gaussian noise to one of the features of the attack samples. The drop in some of the accuracy metric will demonstrate how an attacker can evade an ML-based IDS by conveniently crafting the attack traffic features. 

One approach to increase the robustness of the model to this type of attacks is called Adversarial Training, which consists on training the model with adversarial samples.

In [ ]:
# Author: Roberto Doriguzzi-Corin
# Project: Lecture on Intrusion Detection with Deep Learning
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import copy
import sys
import time
import glob
import pprint
import argparse

import keras.callbacks
import tensorflow as tf
import numpy as np
import random as rn
import os
import csv
import h5py
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Seed Random Numbers
SEED = 1
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)
config = tf.compat.v1.ConfigProto(inter_op_parallelism_threads=1)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense,  Flatten, Conv2D, Reshape, Input, UpSampling2D
from tensorflow.keras.layers import  MaxPooling2D, GlobalMaxPooling2D, Activation
from tensorflow.keras.models import Model, Sequential, save_model, load_model, clone_model
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from scipy.stats import *
from numpy.random import randint, normal
import matplotlib.pyplot as plt

import tensorflow.keras.backend as K
tf.random.set_seed(SEED)
K.set_image_data_format('channels_last')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
#config.log_device_placement = True  # to log device placement (on which device the operation ran)

In [ ]:
FIELDNAMES = ['Model', 'Time', 'Samples', 'DDOS%', 'Accuracy', 'F1Score', 'TPR', 'FPR', 'TNR', 'FNR', 'Source']
FEATURE_INDEX = 1 #number between 0 and 10

# hyperparameters
KERNELS=64
BATCH_SIZE=2048
LEARNING_RATE=0.01

In [ ]:
def load_dataset(path):
    filename = glob.glob(path)[0]
    dataset = h5py.File(filename, "r")
    set_x_orig = np.array(dataset["set_x"][:])  # features
    set_y_orig = np.array(dataset["set_y"][:])  # labels

    X = np.reshape(set_x_orig, (set_x_orig.shape[0], set_x_orig.shape[1], set_x_orig.shape[2], 1))
    Y = set_y_orig

    return X, Y

In [ ]:
def perturb_packet_feature(X, y, feat_index=1, pert_scale=0.01):
    perturbed_X=[]
    for index in range(y.shape[0]):
        x_copy = copy.deepcopy(X[index])
        if y[index] == 1: #we perturb only malicious samples
            for row in x_copy:
                if (np.sum(row) > 0): # we only perturb packets, not padded rows
                    row[feat_index] = np.clip(row[feat_index] + normal(scale=pert_scale),0,1) # add gaussian noise and limit the value in the [0,1] range
            xs = np.squeeze(X[index])
            xcs = np.squeeze(x_copy)
        perturbed_X.append(x_copy)
    return np.array(perturbed_X)

In [ ]:
def plot_pdf(X,y,feat_index=1):
    pdf = []
    total = 0
    for index in range(y.shape[0]):
        x = X[index]
        if y[index] == 1:  # we perturb only malicious samples
            for row in x:
                if (np.sum(row) > 0):  # we only perturb packets, not padded rows
                    feat_value = np.asscalar(row[feat_index])
                    pdf.append(feat_value)
                    total += feat_value
    n, bins, patches = plt.hist(pdf, bins=100)
    plt.show()

In [ ]:
def report_results(Y_true, Y_pred, model_name, data_source, prediction_time):
    ddos_rate = '{:04.3f}'.format(sum(Y_pred) / Y_pred.shape[0])

    if Y_true is not None:  # if we have the labels, we can compute the classification accuracy
        Y_true = Y_true.reshape((Y_true.shape[0], 1))
        accuracy = accuracy_score(Y_true, Y_pred)

        f1 = f1_score(Y_true, Y_pred)
        tn, fp, fn, tp = confusion_matrix(Y_true, Y_pred, labels=[0, 1]).ravel()
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        fnr = fn / (fn + tp)
        tpr = tp / (tp + fn)

        row = {'Model': model_name, 'Time': '{:04.3f}'.format(prediction_time),
               'Samples': Y_pred.shape[0], 'DDOS%': ddos_rate, 'Accuracy': accuracy, 'F1Score': f1,
               'TPR': tpr, 'FPR': fpr, 'TNR': tnr, 'FNR': fnr, 'Source': data_source}
    else:
        row = {'Model': model_name, 'Time': '{:04.3f}'.format(prediction_time),
               'Samples': Y_pred.shape[0], 'DDOS%': ddos_rate, 'Accuracy': "N/A", 'F1Score': "N/A",
               'TPR': "N/A", 'FPR': "N/A", 'TNR': "N/A", 'FNR': "N/A", 'Source': data_source}
    pprint.pprint(row, sort_dicts=False)

## Convolutional Neural Network

In [ ]:
def Conv2DModel(kernels=KERNELS,kernel_rows=3,kernel_col=11,learning_rate=LEARNING_RATE,input_shape=(10,11,1)):
    K.clear_session()

    model = Sequential(name="CNN")
    model.add(Conv2D(kernels, (kernel_rows,kernel_col), strides=(1, 1), input_shape=input_shape, name='conv0'))
    model.add(Activation('relu'))
    model.add(GlobalMaxPooling2D())
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid', name='fc1'))

    compileModel(model, learning_rate)
    print(model.summary())
    return model

In [ ]:
def compileModel(model,lr):
    optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    model.compile(optimizer=optimizer, loss='mean_squared_error')  # here we specify the loss function

## Training phase
Here we train the neural network using a patience value of 10 epochs and a maximum numer of epochs set to 100. You can reduce the value of patience or the max number of epochs is the training process takes too long.

In [ ]:
X_train, y_train = load_dataset('../Datasets/IDS2017/*-train.hdf5')
X_val, y_val = load_dataset('../Datasets/IDS2017/*-val.hdf5')

cnn = Conv2DModel()
cnn.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=100,validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss',restore_best_weights=True, patience=10)])

print("Saving best model's weights...")
cnn.save("./" + cnn.name + ".h5")

In [ ]:
X_test, Y_test = load_dataset('../Datasets/IDS2017/*-test.hdf5')

model_file = glob.glob("." + "/*.h5")[0]
model = load_model(model_file)

## Test phase with unperturbed data
Now we evaluate the trained model on unperturbed test data. Accuracy results are printed on the screen. 

In [ ]:
pt0 = time.time()
Y_pred = np.squeeze(model.predict(X_test, batch_size=2048) > 0.5)
pt1 = time.time()
report_results(Y_test, Y_pred, model.name, "unperturbed attack traffic", pt1 - pt0)

In [ ]:
X_test_perturbed = perturb_packet_feature(X_test, Y_test, FEATURE_INDEX, 0.1)
pt0 = time.time()
Y_pred = np.squeeze(model.predict(X_test_perturbed, batch_size=2048) > 0.5)
pt1 = time.time()
report_results(Y_test, Y_pred, model.name, "perturbed feature " + str(FEATURE_INDEX), pt1 - pt0)

## Plot feature's distribution
We can plot the pdf of the perturbed feature before and after the perturbation to understand how the feature's values have been affected.

In [ ]:
plot_pdf(X_test, Y_test, FEATURE_INDEX)
plot_pdf(X_test_perturbed, Y_test, FEATURE_INDEX)